In [1]:
import torch
class LatencyEncoder:
    def __init__(self, time: int = 100):
        self.time = time  # Общее число временных шагов

    def __call__(self, image: torch.Tensor) -> torch.Tensor:
        """
        image: Tensor [1, 28, 28] или [28, 28], значения от 0 до 1 или до 255
        return: spike_tensor [time, 1, 784]
        """
        if image.ndim == 3:
            image = image.squeeze()

        if image.max() > 1:
            image = image / 255.0

        spike_tensor = torch.zeros((self.time, 1, 784))

        for i in range(28):
            for j in range(28):
                pixel = image[i, j].item()
                if pixel > 0:
                    spike_time = int((1.0 - pixel) * (self.time - 1))
                    spike_tensor[spike_time, 0, i * 28 + j] = 1.0

        return spike_tensor.view(self.time, 1, 28, 28)


In [72]:
def log_input_health(in_s_full, T, name="INPUT", expected_total=None):
    # in_s_full: [T,B,784] или [T,784]
    s = in_s_full[:,0,:] if in_s_full.dim()==3 else in_s_full  # [T,784]
    T_, N = s.shape
    assert T_ == T, f"T mismatch: {T_}!={T}"
    per_pix = s.sum(0)         # [N] спайков на пиксель за окно
    per_t   = s.sum(1)         # [T] спайков на такт

    total  = int(per_pix.sum().item())
    mean_t = float(per_t.mean().item())
    max_t  = int(per_t.max().item())
    frac_silent_pixels = float((per_pix == 0).float().mean().item())
    frac_empty_timesteps = float((per_t == 0).float().mean().item())
    frac_dense_timesteps = float((per_t > 0.1*N).float().mean().item())  # >10% нейронов стреляют

    # Нормированная энтропия распределения активности по пикселям
    if total > 0:
        p = (per_pix / total).clamp_min(1e-12)
        import math
        H = float((-(p * p.log()).sum().item()))           # nats
        H_norm = H / math.log(N)                           # 0..1
    else:
        H_norm = 0.0

    # Топ-8 «самых активных» пикселей (для грубой диагностики)
    topk = torch.topk(per_pix, k=min(8, N))
    topk_vals = [int(v) for v in topk.values.tolist()]

    # Печать
    print(f"{name} window total: {total}"
          + (f" | expected≈{int(expected_total)}" if expected_total is not None else ""))
    print(f"{name} per-timestep: mean={mean_t:.1f}, max={max_t}")
    print(f"{name} pixels: silent_frac={frac_silent_pixels:.3f}, H_norm={H_norm:.3f}")
    print(f"{name} timesteps: empty_frac={frac_empty_timesteps:.3f}, dense>10%={frac_dense_timesteps:.3f}")
    print(f"{name} top-8 pixel counts: {topk_vals}")

    # Возврат на всякий случай (если захочешь логировать в файл)
    return dict(
        total=total, mean_per_timestep=mean_t, max_per_timestep=max_t,
        frac_silent_pixels=frac_silent_pixels, frac_empty_timesteps=frac_empty_timesteps,
        frac_dense_timesteps=frac_dense_timesteps, H_norm=H_norm, top8=topk_vals
    )



In [4]:
def set_stdp_nu(conn, nu_plus, nu_minus):
    dev = conn.w.device
    conn.update_rule.nu = (torch.tensor(nu_plus, device=dev),
                           torch.tensor(nu_minus, device=dev))

In [5]:
# перед run_experiment
class ThreshEMA:
    def __init__(self): self.rate_ema = None
    def step(self, layer, spike_counts, T, target=1.5, alpha=0.9, k=0.02):
        with torch.no_grad():
            rate = spike_counts / max(1, T)
            if self.rate_ema is None:
                self.rate_ema = rate.clone()
            self.rate_ema = alpha * self.rate_ema + (1 - alpha) * rate
            vt = layer.v_thresh if hasattr(layer,'v_thresh') else layer.thresh
            vt += k * (self.rate_ema - target)
            vt.clamp_(0.15, 1.2)
            if hasattr(layer,'v_thresh'): layer.v_thresh = vt
            else: layer.thresh = vt

In [6]:
_rate_ema = None
def adapt_thresholds_ema(layer, spike_counts, T, target=1.5, alpha=0.9, k=0.02):
    global _rate_ema
    with torch.no_grad():
        rate = spike_counts / max(1, T)
        if _rate_ema is None: _rate_ema = rate.clone()
        _rate_ema = alpha * _rate_ema + (1 - alpha) * rate
        vt = layer.v_thresh if hasattr(layer,"v_thresh") else layer.thresh
        vt += k * (_rate_ema - target)
        vt.clamp_(vt_min, vt_max)  
        if hasattr(layer, "v_thresh"): layer.v_thresh = vt
        else: layer.thresh = vt

In [96]:
def _set_param(module, name, value, prefer_scalar=False, fallback_scalar=None):
    """
    Безопасно проставляет module.<name>.
    - Если буфер Tensor скалярный (numel()==1) и value вектор -> кладём 0-D тензор (ср. значение или fallback_scalar).
    - Если буфер Tensor векторный -> копируем по форме.
    - prefer_scalar=True принудительно делает 0-D (для refrac/reset).
    """
    if not hasattr(module, name):
        return False

    cur = getattr(module, name)

    # если это Tensor-буфер
    if isinstance(cur, torch.Tensor):
        dev, dt = cur.device, cur.dtype

        if prefer_scalar:
            # всегда 0-D тензор
            val = float(value.mean().item() if torch.is_tensor(value) else value)
            setattr(module, name, torch.tensor(val, device=dev, dtype=dt))
            return True

        if torch.is_tensor(value):
            if cur.numel() == 1 and value.numel() > 1:
                # буфер скалярный, value вектор -> берём среднее/фолбэк
                val = float(value.mean().item())
                if fallback_scalar is not None:
                    val = float(fallback_scalar)
                setattr(module, name, torch.tensor(val, device=dev, dtype=dt))
            else:
                if value.shape != cur.shape:
                    value = value.view_as(cur)
                cur.data.copy_(value.to(dev, dtype=dt))
        else:
            # value скаляр Python -> просто заливаем
            if cur.numel() == 1:
                setattr(module, name, torch.tensor(float(value), device=dev, dtype=dt))
            else:
                cur.data.fill_(float(value))
        return True

    # не Tensor-буфер – обычный атрибут
    setattr(module, name, float(value) if prefer_scalar else value)
    return True

def debug_all_params(lif_layer):
    print("=== attributes ===")
    for k in dir(lif_layer):
        if not k.startswith("_"):
            try:
                v = getattr(lif_layer, k)
                if torch.is_tensor(v):
                    print(f"{k}: tensor shape={tuple(v.shape)}, first={v.flatten()[0].item():.6g}")
                else:
                    print(f"{k}: {type(v)}")
            except Exception:
                pass

    print("\n=== named_parameters ===")
    for name, p in lif_layer.named_parameters():
        print(f"{name}: shape={tuple(p.shape)}, requires_grad={p.requires_grad}")

    print("\n=== named_buffers ===")
    for name, b in lif_layer.named_buffers():
        print(f"{name}: shape={tuple(b.shape)}")

def tune_lif_params(lif_layer, n_hidden, vt_mean=0.35, vt_jitter=0.02, tau_val=50.0, refrac_val=2.0):
    with torch.no_grad():
        # порог: попытаемся поставить вектор; если буфер скалярный — авто-даунмикс в скаляр
        vt_vec = (vt_mean + vt_jitter * torch.randn(n_hidden)).clamp(0.05, 2.0)
        if not _set_param(lif_layer, "v_thresh", vt_vec, fallback_scalar=vt_mean):
            _set_param(lif_layer, "thresh", vt_vec, fallback_scalar=vt_mean)

        # tau: аналогично (может быть vектор/скаляр в разных версиях)
        if not _set_param(lif_layer, "tc_decay", torch.full((n_hidden,), tau_val), fallback_scalar=tau_val):
            _set_param(lif_layer, "tc_decay",   torch.full((n_hidden,), tau_val),   fallback_scalar=tau_val)

        # refrac — строго скаляр (0-D)
        _set_param(lif_layer, "refrac", refrac_val, prefer_scalar=True)

        # reset — скаляр
        if not _set_param(lif_layer, "v_reset", 0.0, prefer_scalar=True):
            _set_param(lif_layer, "reset",  0.0, prefer_scalar=True)

def print_lif_params(lif_layer):
    with torch.no_grad():
        #debug_all_params(lif_layer)
        # Порог
        for key in ["v_thresh", "thresh"]:
            if hasattr(lif_layer, key):
                val = getattr(lif_layer, key)
                if torch.is_tensor(val):
                    if val.ndim == 0:
                        print(f"[lif] {key} = scalar {val.item():.3f}")
                    else:
                        print(f"[lif] {key} = vector shape={tuple(val.shape)}, "
                              f"mean={val.mean():.3f}, min={val.min():.3f}, max={val.max():.3f}")
                else:
                    print(f"[lif] {key} = {val}")
        
        # Tau
        for key in ["tc_decay"]:
            if hasattr(lif_layer, key):
                val = getattr(lif_layer, key)
                if torch.is_tensor(val):
                    if val.ndim == 0:
                        print(f"[lif] {key} = scalar {val.item():.3f}")
                    else:
                        print(f"[lif] {key} = vector shape={tuple(val.shape)}, "
                              f"value≈{val[0].item():.3f} (all equal?)")
                else:
                    print(f"[lif] {key} = {val}")
        
        # Refrac
        if hasattr(lif_layer, "refrac"):
            val = getattr(lif_layer, "refrac")
            print(f"[lif] refrac = {val if not torch.is_tensor(val) else val.item()}")

        # Reset
        for key in ["v_reset", "reset"]:
            if hasattr(lif_layer, key):
                val = getattr(lif_layer, key)
                print(f"[lif] {key} = {val if not torch.is_tensor(val) else val.item()}")


In [103]:
# ====== Config ======
@dataclass
class Cfg:
    # core
    time:   int = 200
    n_hidden: int = 100
    nu_plus:  float = 0.02
    nu_minus: float = -0.02

    # inhibition / WTA
    inhib_strength: float = 0.3
    inh_decay: float = 0.9
    top_k: int = 0                    # 0 = WTA off for diagnostics
    enable_inhibition_at_start: bool = False

    # encoder
    encoder: str = "latency"         # start with Poisson to "ignite" spikes

    # homeostasis
    target_spikes: float = 2.0
    eta_up: float = 1.0
    eta_down: float = 0.5
    thresh_min: float = 0.2
    thresh_max: float = 2.0
    thresh_init: float = 0.5          # v_thresh initial (BindsNET positive scale)

    # weights
    w_clip_min: float = 0.0
    w_clip_max: float = 1.5
    w_col_target_norm: float = 20.0
    w_init_lo: float = 0.8
    w_init_hi: float = 1.2
    wmin:float = 0.0
    wmax:float = 2.0
    warmup_N: int = 50
    # loop
    N: int = 200
    log_every: int = 50
    seed: int = 42
    poisson_rate_scale: float = 0.7 
    device: str = "cpu"
    vt_mean:float = 0.35
    vt_jitter:float = 0.02
    tau_val:float = 50.0
    refrac_val:float = 2.0
    debug:bool = False

# 🧠 Spiking Neural Network (SNN) на базе BindsNET с обучением через STDP

В этом эксперименте реализована простая биологически правдоподобная спайковая нейросеть (SNN) для обработки изображений MNIST.

## 📌 Архитектура сети:
- **Входной слой (`Input`, 784 нейрона)** — по одному нейрону на каждый пиксель изображения 28×28.
- **Poisson-кодировщик** — преобразует яркость пикселей в вероятностные временные спайки.
- **Полносвязный слой (`Connection`)** — соединяет вход с выходом (матрица весов 784 × 100).
- **Выходной слой (`LIF`, 100 нейронов)** — Leaky Integrate-and-Fire нейроны с утечкой и порогом.
- **STDP (Spike-Timing Dependent Plasticity)** — обучение без градиентов; веса усиливаются, если вход активен до выходного спайка.

## 🔬 Что делается:
1. Загружается одно изображение MNIST.
2. Кодируется в Poisson-спайковый поток.
3. Пропускается через сеть:
   - `Input` получает входные спайки,
   - `LIF` нейроны активируются в зависимости от весов.
4. Сохраняются:
   - Спайковая активность `LIF`-нейронов до и после подачи входа.
   - Сумма спайков на входе (`Input`) — показывает, какие пиксели активны.
   - Веса одного выбранного `LIF`-нейрона — до и после STDP.

## 📈 Визуализация:
- График: сравнение спайковой активности нейронов до/после + входные спайки.
- График: изменение весов, ведущих к нейрону `LIF[42]` — видно, как STDP усиливает значимые связи.

## 🎯 Цель:
Показать, как SNN:
- преобразует изображение в поток спайков,
- активирует только специфичные нейроны,
- адаптирует веса на основе временных шаблонов (STDP), без использования обратного распространения ошибки.


In [104]:
# ====== SETUP (Colab/Local) ======
# !pip -q install bindsnet==0.2.8 torchvision==0.18.1 torch==2.3.1 --extra-index-url https://download.pytorch.org/whl/cu121

import os, itertools, random, csv, time as _ptime
from dataclasses import dataclass, asdict

import torch
import numpy as np
import matplotlib.pyplot as plt

# ====== Utils ======
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)

def to_2d(s):  # [T,N] or [T,1,N] -> [T,N]
    return s[:,0,:] if (s.dim()==3 and s.size(1)==1) else s



# ====== Helpers: WTA, norm, thresholds, plots, metrics ======
def apply_wta(s, top_k=1):
    s2 = to_2d(s)
    sb = s2.sum(0).float().squeeze()
    if sb.sum() == 0:
        return False, None
    vals, idxs = torch.topk(sb, k=min(top_k, sb.numel()))
    s.zero_()
    for j in idxs.tolist():
        if s.dim()==3:
            s[:,0,j] = True
        else:
            s[:,j] = True
    return True, idxs.tolist()

def weight_soft_bound_and_colnorm(conn_w, w_clip_min, w_clip_max, target_norm):
    with torch.no_grad():
        w = conn_w.data
        w.clamp_(w_clip_min, w_clip_max)
        col_norm = w.norm(p=1, dim=0, keepdim=True) + 1e-6
        w.mul_(target_norm / col_norm)

def adapt_thresholds(layer, spike_counts, cfg: Cfg):
    with torch.no_grad():
        vt = layer.v_thresh if hasattr(layer, "v_thresh") else layer.thresh
        vt -= 0.05 * (spike_counts < 1.0).float()        # if silent -> lower threshold
        vt += 0.02 * (spike_counts > 3.0).float()        # if too active -> raise
        vt.clamp_(cfg.thresh_min, cfg.thresh_max)
        if hasattr(layer, "v_thresh"): layer.v_thresh = vt
        else: layer.thresh = vt

def spiking_metrics_window(lif_s, winners=None):
    s = to_2d(lif_s).to(torch.bool)
    T, N = s.shape
    per_n = s.sum(0)
    tot = int(per_n.sum())
    active = int((per_n > 0).sum())
    if tot > 0:
        p = (per_n / tot).float().cpu().numpy()
        HHI = float((p**2).sum())
        ps = np.sort(p)
        Gini = float((np.cumsum(ps).sum()/ps.sum() - (len(ps)+1)/2)/len(ps))
    else:
        HHI, Gini = 1.0, 1.0
    uniq_winners = len(set(winners)) if winners else 0
    return dict(T=T, N=N, total_spikes=tot, active=active, HHI=HHI, Gini=Gini, uniq_winners=uniq_winners)

class SNNMeter:
    def __init__(self): self.reset()
    def reset(self):
        self.samples=0; self.S_out=0; self.S_in=0; self.SynOps=0; self.V_updates=0
        self.usage_counts = {}
    def log_sample(self, lif_s, in_s, n_hidden, T, winners=None):
        lif2 = to_2d(lif_s);  in2 = to_2d(in_s)
        s_out = int(lif2.sum().item())
        s_in  = int(in2.sum().item())
        self.S_out += s_out; self.S_in += s_in
        self.SynOps += s_in * n_hidden
        self.V_updates += n_hidden * T
        self.samples += 1
        if winners:
            for j in winners:
                self.usage_counts[j] = self.usage_counts.get(j,0)+1
    def report(self, a=1.0, b=0.05, c=0.005):
        s = max(1, self.samples)
        HHI_win = 0.0
        if self.usage_counts:
            tot = sum(self.usage_counts.values())
            ps = np.array([v/tot for v in self.usage_counts.values()], dtype=float)
            HHI_win = float((ps**2).sum())
        return {
            "spikes_per_sample": self.S_out/s,
            "synops_per_sample": self.SynOps/s,
            "v_updates_per_sample": self.V_updates/s,
            "energy_proxy_per_sample": (a*self.S_out + b*self.SynOps + c*self.V_updates)/s,
            "winners_unique": len(self.usage_counts),
            "winner_HHI": HHI_win,
        }

# ====== Build Net & Encoder ======
from bindsnet.network import Network
from bindsnet.network.nodes import Input, LIFNodes
from bindsnet.network.topology import Connection
from bindsnet.learning import PostPre
from torchvision import transforms
from bindsnet.datasets import MNIST
from bindsnet.network.monitors import Monitor

def build_net(cfg: Cfg):
    net = Network()

    input_layer = Input(n=784, traces=True)
    lif_layer   = LIFNodes(n=cfg.n_hidden, traces=True)
    tune_lif_params(lif_layer, cfg.n_hidden, cfg.vt_mean, cfg.vt_jitter, cfg.tau_val, cfg.refrac_val)
            
    net.add_layer(input_layer, name='Input')
    net.add_layer(lif_layer,   name='LIF')
    print_lif_params(lif_layer)

    connection = Connection(source=input_layer, target=lif_layer)
    connection.update_rule = PostPre(connection=connection,
                                 nu=(torch.tensor(cfg.nu_plus),
                                     torch.tensor(cfg.nu_minus)))
    net.add_connection(connection, source='Input', target='LIF')

    # Lateral inhibition (created, but optionally disabled at start)
    W_inh = torch.full((cfg.n_hidden, cfg.n_hidden), -cfg.inhib_strength)
    W_inh.fill_diagonal_(0.0)
    recurrent_inh = Connection(source=lif_layer, target=lif_layer, w=W_inh.clone())
    net.add_connection(recurrent_inh, source='LIF', target='LIF')

    # Weights init (stronger to ignite)
    with torch.no_grad():
        connection.w.data.uniform_(cfg.w_init_lo, cfg.w_init_hi)

    # Thresholds: use v_thresh if available
    th0 = torch.full((cfg.n_hidden,), cfg.thresh_init)
    if hasattr(lif_layer, "v_thresh"): lif_layer.v_thresh = th0.clone()
    else: lif_layer.thresh = th0.clone()

    # Optionally disable inhibition at start (for diagnostics)
    if not cfg.enable_inhibition_at_start:
        with torch.no_grad():
            recurrent_inh.w.zero_()

    return net, input_layer, lif_layer, connection, recurrent_inh, W_inh

# Пре-процесс для произвольных изображений (PIL / np / tensor)
preprocess = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28), antialias=True),
    transforms.ToTensor(),                 # -> [1,28,28] float in [0,1]
    # Никаких Normalize(mean,std) здесь — нам нужны «сырые» 0..1!
])

def make_encoder(encoder_type: str, T: int, rate_floor: float = 0.0,poisson_rate_scale: float = 0.7 ):  # floor по умолчанию 0
    encoder_type = encoder_type.lower()
    assert encoder_type in ("poisson", "latency")

    def encode_poisson(img_tensor):
        x = img_tensor.view(-1).clamp(0, 1)
        rates = x * poisson_rate_scale
        rand = torch.rand((T, rates.numel()), device=rates.device if x.is_cuda else None)
        spikes = (rand < rates).float()
        return spikes.view(T, 1, 784)

    def encode_latency(img_tensor):
        x = img_tensor.squeeze(0).clamp(0, 1)
        spikes = torch.zeros((T, 1, 784), dtype=torch.float32)
        nz = (x > 0).nonzero(as_tuple=False)
        if nz.numel() == 0:
            return spikes
        for idx in nz:
            i, j = int(idx[0]), int(idx[1])
            p = float(x[i, j])
            t = int(round((1.0 - p) * (T - 1)))
            # маленький джиттер ±1 тик (в пределах окна)
            if T >= 3:
                t += int(torch.randint(-1, 2, (1,)).item())
                t = max(0, min(T-1, t))
            spikes[t, 0, i*28 + j] = 1.0
        return spikes

    return (encode_poisson if encoder_type == "poisson" else encode_latency), preprocess
        
def _to_2d(s):  # [T, B, N] -> [T, N]
    return s[:, 0, :] if s.dim()==3 else s
    
# ====== One Experiment ======
def run_experiment(cfg: Cfg, verbose=True):
    set_seed(cfg.seed)
    device = cfg.device
    print("Используем:", device)
    # Датасет (MNIST уже в [0,1] и [1,28,28])
    transform = transforms.Compose([transforms.ToTensor()])
    dataset = MNIST(root='./data', train=True, download=True, transform=transform)
    # Сеть + мониторы
    net, input_layer, lif_layer, connection, recurrent_inh, W_inh = build_net(cfg)
    net = net.to(device)
   
    
    
    

    # ↓ СРАЗУ ПОСЛЕ build_net(cfg)
    with torch.no_grad():
        # у разных версий BindsNET порог лежит в v_thresh ИЛИ в thresh
        if hasattr(lif_layer, "v_thresh"):
            vt = lif_layer.v_thresh
            if isinstance(vt, torch.Tensor) and vt.numel() == 1:
                lif_layer.v_thresh = torch.tensor(0.12, device=vt.device, dtype=vt.dtype)  # 0-D тензор!
            else:
                lif_layer.v_thresh.fill_(0.12)
        else:
            vt = lif_layer.thresh
            if isinstance(vt, torch.Tensor) and vt.numel() == 1:
                lif_layer.thresh = torch.tensor(0.12, device=vt.device, dtype=vt.dtype)   # 0-D тензор!
            else:
                lif_layer.thresh.fill_(0.12)
        if hasattr(lif_layer, "refrac"):
            lif_layer.refrac = torch.tensor(2.0, device=vt.device)  # 2 тика
            #print(f"set refrac {lif_layer.refrac}")
            
    
    # для самопроверки — оставь print один раз
    vt_chk = (lif_layer.v_thresh if hasattr(lif_layer,"v_thresh") else lif_layer.thresh)
    #print(">>> THRESH SET TO:", float(vt_chk.mean().item()))
    vt =  (lif_layer.v_thresh if hasattr(lif_layer,"v_thresh") else lif_layer.thresh)
    rf = lif_layer.refrac
    #print("v_thresh mean±std refrac:", float(vt.mean()), float(vt.std()),rf)
    
    lif_mon = Monitor(lif_layer, state_vars=("s",), time=cfg.time)
    inp_mon = Monitor(input_layer, state_vars=("s",), time=cfg.time)
    net.add_monitor(lif_mon, name="lif_mon")
    net.add_monitor(inp_mon, name="inp_mon")

    # Энкодер
    ENCODER_TYPE = cfg.encoder
    T = cfg.time
    encoder, _ = make_encoder(ENCODER_TYPE, T, poisson_rate_scale=cfg.poisson_rate_scale)

    # --- DEBUG METRICS INIT --------------------------------------------
    DEBUG = cfg.debug
    pix_total = 28 * 28
    # аккумулируем статистику по всему ранe
    inp_empty_steps_total = 0     # сколько тайм-степов вообще пустые (нет входных спайков)
    lif_empty_steps_total = 0
    steps_total = 0
    pix_fire_counts = torch.zeros(pix_total)  # суммарные спайки по каждому пикселю
    run_pairs = []  # (sum_in_window, sum_lif_window)
    # --------------------------------------------------------------------
    
    # --------- WARMUP (без STDP) ---------
    WARMUP = getattr(cfg, "warmup_N", 50)
    if WARMUP > 0:
        # на прогрев STDP выкл.
        set_stdp_nu(connection, 0.0, 0.0)
        for wi in range(min(WARMUP, len(dataset))):
            image = dataset[wi]["image"]
             # === ВСТАВИТЬ В ЦИКЛ ПЕРЕД net.run(...) ===
            # Оценка ожидаемого числа входных спайков (для Poisson)
            if cfg.encoder.lower() == "poisson":
                # Если в Poisson ты делаешь: spikes[t] ~ Bernoulli(rate_scale * pixel)
                img_mean = float(image.mean().item())  # [0..1]
                expected_total = cfg.time * 784 * cfg.poisson_rate_scale * img_mean
            else:
                expected_total = None
            spike_input = encoder(image).to(device)
            net.run(inputs={"Input": spike_input}, time=cfg.time)

            # адаптация порогов на прогреве (по желанию — полезно)
            lif_s_full = lif_mon.get("s")
            spike_counts = to_2d(lif_s_full).sum(0).float().squeeze()
            # adapt_thresholds_ema(lif_layer, spike_counts, cfg.time, target=1.5)
            

            # очистка состояний между примерами
            net.reset_state_variables()
            lif_mon.reset_state_variables()
            inp_mon.reset_state_variables()

    vt =  (lif_layer.v_thresh if hasattr(lif_layer,"v_thresh") else lif_layer.thresh)
    #print("v_thresh mean±std:", float(vt.mean()), float(vt.std()))
    # --------- ОСНОВНОЙ ЦИКЛ ---------
    with torch.no_grad():
        I = torch.eye(cfg.n_hidden, device=recurrent_inh.w.device, dtype=recurrent_inh.w.dtype)
        recurrent_inh.w.copy_(-0.55 * (1 - I)) 
    ema = ThreshEMA()
    meter = SNNMeter()
    # мягкий STDP после прогрева
    set_stdp_nu(connection, cfg.nu_plus, cfg.nu_minus) # было 1e-3 / -5e-4
    

    for i in range(cfg.N):
        sample = dataset[i]
        image  = sample["image"]
        spike_input = encoder(image).to(device)
        inputs = {"Input": spike_input}
        

        net.run(inputs=inputs, time=cfg.time)

        # Полный растр за окно
        lif_s_full = lif_mon.get("s")   # [T,B,N]
        in_s_full  = inp_mon.get("s")   # [T,B,784]
        lif2 = _to_2d(lif_s_full); in2 = _to_2d(in_s_full)

        # --- DEBUG METRICS PER-WINDOW ---------------------------------
        # по-времени: сколько событий на каждом t
        inp_t = in2.sum(dim=1).cpu().numpy()   # [T]
        lif_t = lif2.sum(dim=1).cpu().numpy()  # [T]
        steps_total += T
        inp_empty_steps_total += int((inp_t == 0).sum())
        lif_empty_steps_total += int((lif_t == 0).sum())
        # по-пикселям: сколько раз пиксель стрелял за окно
        pix_fire_counts += in2.sum(dim=0).cpu()  # [784]
        # окно-суммы для быстрой корреляции «окно входа» → «окно выхода»
        run_pairs.append((float(inp_t.sum()), float(lif_t.sum())))
        # --------------------------------------------------------------

        # чекпоинт-лог (минимальный)
        if i in (0, 50, 100, 150, 199) or ((i+1) % cfg.log_every == 0):
            print(f"INPUT window sum: {int(inp_t.sum())}")
            print(f"LIF   window sum: {int(lif_t.sum())}")

            if DEBUG:
                # доля пустых таймстепов в текущем окне
                empty_frac_in  = float((inp_t == 0).mean())
                empty_frac_lif = float((lif_t == 0).mean())
                mean_in_t  = float(inp_t.mean());  max_in_t  = float(inp_t.max(initial=0))
                mean_lif_t = float(lif_t.mean());  max_lif_t = float(lif_t.max(initial=0))

                # корреляция по тайм-ось: сколько входных событий ↔ сколько выходных
                if (inp_t.std() > 1e-8) and (lif_t.std() > 1e-8):
                    corr = float(np.corrcoef(inp_t, lif_t)[0,1])
                else:
                    corr = float('nan')

                print(f"[dbg] per-timestep INPUT mean={mean_in_t:.2f} max={max_in_t} empty_frac={empty_frac_in:.2f}")
                print(f"[dbg] per-timestep LIF   mean={mean_lif_t:.2f} max={max_lif_t} empty_frac={empty_frac_lif:.2f}")
                print(f"[dbg] corr(input_t, lif_t)={corr:.3f}")

                # топ-8 активных пикселей (индикатор, что вход не совсем «немой»)
                topk = torch.topk(pix_fire_counts, k=8)
                print(f"[dbg] top8 pixels counts: {topk.values.int().tolist()}  idx: {topk.indices.tolist()[:8]}")


        # WTA (если включён)
        winners = []
        if cfg.top_k and cfg.top_k > 0:
            ok, idxs = apply_wta(lif_layer.s, top_k=cfg.top_k)
            winners = idxs if ok and idxs is not None else []

        # Метрики окна
        m = spiking_metrics_window(lif_s_full, winners)
        if verbose and ((i+1) % cfg.log_every == 0 or i == 0):
            print(f"[{i+1}] total={m['total_spikes']} active={m['active']}/{m['N']} HHI={m['HHI']:.3f}")

        # Homeostasis по «сырым» спайкам (до WTA)
        spike_counts = to_2d(lif_s_full).sum(0).float().to(device)
        ema.step(lif_layer, spike_counts, cfg.time, target=1.5)

        # Кламп весов (без агрессивной колоночной нормировки каждый шаг)
        with torch.no_grad():
            connection.w.clamp_(0.0, 1.0)

        # Энергетика / учёт
        meter.log_sample(lif_s_full, in_s_full, cfg.n_hidden, cfg.time, winners=winners)

        # Сброс состояний и мониторов
        net.reset_state_variables()
        lif_mon.reset_state_variables()
        inp_mon.reset_state_variables()

        
    # --- DEBUG METRICS FINAL SUMMARY -----------------------------------
    if DEBUG and steps_total > 0:
        inp_empty_frac_all = inp_empty_steps_total / steps_total
        lif_empty_frac_all = lif_empty_steps_total / steps_total
        silent_pixels_frac = float((pix_fire_counts == 0).float().mean())

        # корреляция «сумма входа по окну → сумма выхода по окну»
        if len(run_pairs) > 1:
            inp_sums = np.array([p[0] for p in run_pairs], dtype=float)
            lif_sums = np.array([p[1] for p in run_pairs], dtype=float)
            if inp_sums.std() > 1e-8 and lif_sums.std() > 1e-8:
                corr_win = float(np.corrcoef(inp_sums, lif_sums)[0,1])
            else:
                corr_win = float('nan')
        else:
            corr_win = float('nan')

        q = torch.quantile(pix_fire_counts, torch.tensor([0.0, 0.5, 0.9, 0.99]))
        print("\n=== INPUT DENSITY SUMMARY ===")
        print(f"empty_frac_all (per-timestep): INPUT={inp_empty_frac_all:.2f} | LIF={lif_empty_frac_all:.2f}")
        print(f"silent_pixels_frac: {silent_pixels_frac:.2f}")
        print(f"pix_fire_counts quantiles [min,median,p90,p99]: {q.tolist()}")
        print(f"corr(window_sum_in, window_sum_lif): {corr_win:.3f}")
    # --------------------------------------------------------------------
   
    rpt = meter.report()
    if getattr(meter, "samples", 0) == 0:
        print("!! meter: no samples logged — проверь порядок log_sample()/reset() и continue в цикле")
    out = {**asdict(cfg), **rpt}
    return out, connection, lif_layer

# ====== Grid Runner (compact) ======



In [ ]:
cfg = Cfg(
    time=200,
    n_hidden=100,
    encoder="poisson",                 # сначала Poisson
    top_k=3,                           # WTA выкл. для диагностики
    enable_inhibition_at_start=False,  # ингибицию включим позже
   
    poisson_rate_scale = 0.006
)
res = run_experiment(cfg, verbose=True)
print("\nSUMMARY:", {k: res[k] for k in ["spikes_per_sample","winners_unique","winner_HHI","energy_proxy_per_sample"]})


In [ ]:
from dataclasses import asdict

# Сетка значений для сканирования
scales = [0.002, 0.004, 0.006, 0.01, 0.02, 0.05, 0.1, 0.2]

results = []
print("scan poisson_rate_scale → [spikes/sample, winners_unique, HHI, energy_proxy]\n")
for s in scales:
    cfg_s = Cfg(**{**asdict(cfg), "poisson_rate_scale": s})
    res = run_experiment(cfg_s, verbose=False)
    results.append(res)
    print(f"{s:>6}: {res['spikes_per_sample']:.2f}, "
          f"{res['winners_unique']}, "
          f"{res['winner_HHI']:.3f}, "
          f"{res['energy_proxy_per_sample']:.1f}")

# Простейший отбор «разумных» настроек:
#   - хотим winners_unique > 0 (есть специализация)
#   - хотим умеренную активность (не лавина): spikes_per_sample в [20, 400] (подправь под свою цель)
#   - минимизируем energy_proxy_per_sample
candidates = [
    r for r in results
    if r["winners_unique"] > 0 and 20 <= r["spikes_per_sample"] <= 400
]
if candidates:
    best = min(candidates, key=lambda r: r["energy_proxy_per_sample"])
    print("\nBEST (by lowest energy among reasonable activity):")
    print({k: best[k] for k in ["poisson_rate_scale","spikes_per_sample",
                                "winners_unique","winner_HHI","energy_proxy_per_sample"]})
else:
    print("\nNo reasonable candidates found — relax constraints or widen scales.")


In [51]:
from tqdm import tqdm
import itertools, csv, math, traceback

# ---- настрой сетку здесь ----
param_grid = {
    "poisson_rate_scale": [ 0.006, 0.007, 0.008, 0.009, 0.01],
    "nu_plus":            [1e-4, 3e-4, 1e-3, 3e-3],
    "nu_minus":           [-5e-5, -1e-4, -3e-4, -1e-3],  # ОТРИЦАТЕЛЬНЫЕ
    "top_k":              [0, 1, 3, 4, 5],
    # если захочешь — добавь сюда "time", "n_hidden", но тогда меняй сборку cfg ниже
}

def grid_search(param_grid, out_csv="grid_results.csv", seed=42, verbose_every=0):
    keys = list(param_grid.keys())
    vals = [param_grid[k] for k in keys]
    total = 1
    for v in vals: total *= len(v)
    print(f"Комбинаций: {total}")

    # CSV
    header = keys + [
        "spikes_per_sample",
        "winners_unique",
        "winner_HHI",
        "energy_proxy_per_sample",
    ]
    f = open(out_csv, "w", newline="")
    writer = csv.writer(f); writer.writerow(header)

    best = []  # будем хранить топ-5
    def score(res):
        # цель: больше специализации, меньше энергии и лишних спайков
        # комбинированный ключ: (-winners_unique, winner_HHI возм., energy, spikes)
        # но для сортировки возьмём tuple (энергия, -winners_unique, winner_HHI)
        return (res["energy_proxy_per_sample"], -res["winners_unique"], res["winner_HHI"])

    with tqdm(total=total, desc="Grid search") as pbar:
        for combo in itertools.product(*vals):
            cfg_dict = dict(zip(keys, combo))
            try:
                cfg = Cfg(
                    time=200,
                    n_hidden=100,
                    encoder="poisson",
                    top_k=int(cfg_dict["top_k"]),
                    enable_inhibition_at_start=False,
                    nu_plus=float(cfg_dict["nu_plus"]),
                    nu_minus=float(cfg_dict["nu_minus"]),           # отрицательные допустимы
                    poisson_rate_scale=float(cfg_dict["poisson_rate_scale"]),
                    seed=seed,
                )

                res = run_experiment(cfg, verbose=False)
                row = [cfg_dict[k] for k in keys] + [
                    res["spikes_per_sample"],
                    res["winners_unique"],
                    res["winner_HHI"],
                    res["energy_proxy_per_sample"],
                ]
                writer.writerow(row); f.flush()

                # обновить топ-5
                best.append(res)
                best.sort(key=score)
                if len(best) > 5: best = best[:5]

            except Exception as e:
                # логируем «плохую» точку
                row = [cfg_dict[k] for k in keys] + ["ERROR", "ERROR", "ERROR", "ERROR"]
                writer.writerow(row); f.flush()
                print("\n[WARN] Ошибка на комбе:", cfg_dict)
                traceback.print_exc()

            pbar.update(1)

    f.close()

    print("\nTop-5 (по энерго-метрике с приоритетом специализации):")
    for i, r in enumerate(best, 1):
        short = {
            "poisson_rate_scale": r.get("poisson_rate_scale", None) if isinstance(r.get("poisson_rate_scale", None), (int,float)) else None,
            "nu_plus": r.get("nu_plus", None) if isinstance(r.get("nu_plus", None), (int,float)) else None,
            "nu_minus": r.get("nu_minus", None) if isinstance(r.get("nu_minus", None), (int,float)) else None,
            "top_k": r.get("top_k", None) if isinstance(r.get("top_k", None), (int,float)) else None,
            "spikes_per_sample": r["spikes_per_sample"],
            "winners_unique": r["winners_unique"],
            "winner_HHI": r["winner_HHI"],
            "energy_proxy_per_sample": r["energy_proxy_per_sample"],
        }
        print(f"{i}.", short)

    print(f"\nСохранено: {out_csv}")


In [53]:
# ---- настрой сетку здесь ----
param_grid_set2 = {
    "poisson_rate_scale": [0.004, 0.006, 0.008],
    "nu_plus": [0.001, 0.002, 0.003],
    "nu_minus": [-0.0005, -0.001, -0.002],
    "top_k": [0, 3, 5]
}
param_grid_set1 = {
    "poisson_rate_scale": [ 0.006, 0.007, 0.008],
    "nu_plus":            [1e-4, 3e-4, 1e-3, 3e-3],
    "nu_minus":           [-5e-5, -1e-4, -3e-4, -1e-3],  # ОТРИЦАТЕЛЬНЫЕ
    "top_k":              [0,  3,  5,6],
    # если захочешь — добавь сюда "time", "n_hidden", но тогда меняй сборку cfg ниже
}
param_grid_set3 = {
    "poisson_rate_scale": [ 0.006, 0.065,],
    "nu_plus":            [0.0001],
    "nu_minus":           [-0.001],  # ОТРИЦАТЕЛЬНЫЕ
    "top_k":              [ 5,6,7,8],
    # если захочешь — добавь сюда "time", "n_hidden", но тогда меняй сборку cfg ниже
}

grid_search(param_grid_set3, out_csv="grid_results_set3.csv")

Комбинаций: 8


Grid search: 100%|████████████████████████████████████████████████████████████████████████| 8/8 [04:54<00:00, 36.82s/it]


Top-5 (по энерго-метрике с приоритетом специализации):
1. {'poisson_rate_scale': 0.006, 'nu_plus': 0.0001, 'nu_minus': -0.001, 'top_k': 8, 'spikes_per_sample': 18.885, 'winners_unique': 35, 'winner_HHI': 0.036458333333333336, 'energy_proxy_per_sample': 711.21}
2. {'poisson_rate_scale': 0.006, 'nu_plus': 0.0001, 'nu_minus': -0.001, 'top_k': 7, 'spikes_per_sample': 18.885, 'winners_unique': 32, 'winner_HHI': 0.03854875283446711, 'energy_proxy_per_sample': 711.21}
3. {'poisson_rate_scale': 0.006, 'nu_plus': 0.0001, 'nu_minus': -0.001, 'top_k': 6, 'spikes_per_sample': 18.885, 'winners_unique': 28, 'winner_HHI': 0.04166666666666667, 'energy_proxy_per_sample': 711.21}
4. {'poisson_rate_scale': 0.006, 'nu_plus': 0.0001, 'nu_minus': -0.001, 'top_k': 5, 'spikes_per_sample': 18.885, 'winners_unique': 25, 'winner_HHI': 0.04666666666666667, 'energy_proxy_per_sample': 711.21}
5. {'poisson_rate_scale': 0.065, 'nu_plus': 0.0001, 'nu_minus': -0.001, 'top_k': 8, 'spikes_per_sample': 1419.19, 'winners_

Top-5 (по энерго-метрике с приоритетом специализации):
1. {'poisson_rate_scale': 0.004, 'nu_plus': 0.003, 'nu_minus': -0.002, 'top_k': 0, 'spikes_per_sample': 0.195, 'winners_unique': 0, 'winner_HHI': 0.0, 'energy_proxy_per_sample': 492.995}
2. {'poisson_rate_scale': 0.004, 'nu_plus': 0.003, 'nu_minus': -0.002, 'top_k': 3, 'spikes_per_sample': 0.195, 'winners_unique': 0, 'winner_HHI': 0.0, 'energy_proxy_per_sample': 492.995}
3. {'poisson_rate_scale': 0.004, 'nu_plus': 0.003, 'nu_minus': -0.002, 'top_k': 5, 'spikes_per_sample': 0.195, 'winners_unique': 0, 'winner_HHI': 0.0, 'energy_proxy_per_sample': 492.995}
4. {'poisson_rate_scale': 0.004, 'nu_plus': 0.001, 'nu_minus': -0.0005, 'top_k': 0, 'spikes_per_sample': 0.2, 'winners_unique': 0, 'winner_HHI': 0.0, 'energy_proxy_per_sample': 493.0}
5. {'poisson_rate_scale': 0.004, 'nu_plus': 0.001, 'nu_minus': -0.0005, 'top_k': 3, 'spikes_per_sample': 0.2, 'winners_unique': 0, 'winner_HHI': 0.0, 'energy_proxy_per_sample': 493.0}

In [54]:
import os, json, torch
from torch.utils.data import Subset
from torchvision import transforms
from bindsnet.datasets import MNIST
from bindsnet.network.monitors import Monitor

# ====== 1) SAVE / LOAD ======
def save_snn(path, cfg, connection, lif_layer):
    os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
    vt = (lif_layer.v_thresh if hasattr(lif_layer,'v_thresh') else lif_layer.thresh)
    ckpt = {
        "cfg": asdict(cfg),
        "W": connection.w.detach().cpu(),
        "v_thresh": vt.detach().cpu(),
    }
    torch.save(ckpt, path)
    print(f"Saved to {path} | W {tuple(ckpt['W'].shape)}")

def load_weights_into(net, connection, lif_layer, ckpt_path):
    ckpt = torch.load(ckpt_path, map_location="cpu")
    with torch.no_grad():
        connection.w.copy_(ckpt["W"])
        vt = ckpt["v_thresh"]
        if hasattr(lif_layer, "v_thresh"): lif_layer.v_thresh.copy_(vt)
        else: lif_layer.thresh.copy_(vt)
    print(f"Loaded from {ckpt_path}")

# ====== 2) КАЛИБРОВКА (нейрон -> метка) ======
@torch.no_grad()
def build_label_map(net, input_layer, lif_layer, encoder, n_calib=2000, T=200, top_k=3, seed=123):
    # выключаем обучение
    for c in net.connections.values():
        if hasattr(c, "update_rule"): c.update_rule.nu = (torch.as_tensor(0.0), torch.as_tensor(0.0))

    lif_mon = Monitor(lif_layer, state_vars=("s",), time=T); net.add_monitor(lif_mon, name="lif_eval_tmp")

    transform = transforms.Compose([transforms.ToTensor()])
    ds_train = MNIST(root="./data", train=True, download=True, transform=transform)
    idxs = list(range(min(n_calib, len(ds_train))))
    usage = torch.zeros((lif_layer.n,), dtype=torch.long)          # сколько раз нейрон выигрывал
    wins  = torch.zeros((lif_layer.n, 10), dtype=torch.long)       # нейрон x класс

    for i in idxs:
        torch.manual_seed(seed + i)  # фиксируем стохастику Poisson per-sample
        x = ds_train[i]["image"]
        y = int(ds_train[i]["label"])
        spikes_in = encoder(x)                       # [T,1,784]
        net.run(inputs={"Input": spikes_in}, time=T)

        # выбираем победителей по сумме спайков за окно
        s = lif_mon.get("s")                         # [T,1,N]
        s2 = s[:,0,:]                                # [T,N]
        counts = s2.sum(0)                           # [N]
        if counts.sum() > 0:
            k = min(top_k, lif_layer.n)
            topv, topi = torch.topk(counts, k=k)
            for j in topi.tolist():
                usage[j] += 1
                wins[j, y] += 1

        net.reset_state_variables()
        lif_mon.reset_state_variables()

    net.monitors.pop("lif_eval_tmp", None)

    # нейронная метка = argmax по классам (если нейрон хоть раз выигрывал)
    label_map = -torch.ones((lif_layer.n,), dtype=torch.long)
    active = (usage > 0).nonzero().flatten().tolist()
    for j in active:
        label_map[j] = wins[j].argmax().item()

    covered = int((label_map >= 0).sum())
    print(f"Label-map built: {covered}/{lif_layer.n} neurons assigned; active winners {int((usage>0).sum())}")
    return label_map

# ====== 3) ОЦЕНКА НА TEST ======
@torch.no_grad()
def evaluate_on_mnist(net, input_layer, lif_layer, encoder, label_map, T=200, top_k=3, n_test=1000, seed=999):
    # freeze learning
    for c in net.connections.values():
        if hasattr(c, "update_rule"): c.update_rule.nu = (torch.as_tensor(0.0), torch.as_tensor(0.0))

    lif_mon = Monitor(lif_layer, state_vars=("s",), time=T); net.add_monitor(lif_mon, name="lif_test_tmp")
    transform = transforms.Compose([transforms.ToTensor()])
    ds_test = MNIST(root="./data", train=False, download=True, transform=transform)
    idxs = list(range(min(n_test, len(ds_test))))

    correct = 0
    meter = SNNMeter()

    for i in idxs:
        torch.manual_seed(seed + i)
        x = ds_test[i]["image"]; y = int(ds_test[i]["label"])
        spikes_in = encoder(x)
        net.run(inputs={"Input": spikes_in}, time=T)

        s_full = lif_mon.get("s")              # [T,1,N]
        s2 = s_full[:,0,:]                     # [T,N]
        counts = s2.sum(0)                     # [N]

        # WTA на оценке — берём top_k нейронов и голосуем их метками
        k = min(top_k, lif_layer.n)
        if counts.sum() == 0:
            pred = -1
        else:
            topv, topi = torch.topk(counts, k=k)
            votes = torch.zeros(10, dtype=torch.float32)
            for j, v in zip(topi.tolist(), topv.tolist()):
                lbl = int(label_map[j].item())
                if lbl >= 0: votes[lbl] += float(v)
            pred = int(votes.argmax().item()) if votes.sum() > 0 else -1

        if pred == y: correct += 1

        # энергетика (для контроля)
        # создадим фиктивный “input monitor” из тех же спайков
        meter.log_sample(s_full, spikes_in, lif_layer.n, T, winners=topi.tolist() if counts.sum()>0 else None)

        net.reset_state_variables()
        lif_mon.reset_state_variables()

    acc = correct / len(idxs)
    rpt = meter.report()
    net.monitors.pop("lif_test_tmp", None)
    print(f"TEST accuracy: {acc:.3f}  | spikes/sample={rpt['spikes_per_sample']:.2f}  energy≈{rpt['energy_proxy_per_sample']:.1f}")
    return {"accuracy": acc, **rpt}




In [107]:
# ====== ПРИМЕР ИСПОЛЬЗОВАНИЯ ======
# 1) тренируем как раньше:
cfg = Cfg(
    time = 200,                 # число тактов симуляции на образ
    n_hidden = 100    ,         # количество скрытых нейронов
    encoder = "poisson",        # кодировщик входа: пуассоновский
    top_k = 3           ,       # количество победителей в WTA
    enable_inhibition_at_start = False,  # включать ли торможение сразу
    nu_plus = 0.0001,           # шаг LTP при STDP
    nu_minus = -0.001 ,         # шаг LTD при STDP
    poisson_rate_scale = 0.006, # масштаб интенсивности пуассоновских потоков
    device = "cpu",             # устройство вычислений
    log_every = 50 ,            # логирование каждые N шагов/образов
    N = 1000   ,                # размер датасета
    vt_mean = 0.35,             # средний порог возбуждения нейронов
    vt_jitter = 0.02   ,        # разброс порогов по нейронам
    tau_val = 150.0   ,          # константа времени утечки мембраны (aka tc_decay)
    refrac_val = 2.0  ,         # рефрактерный период после спайка
    debug = False
)

res, connection, lif_layer = run_experiment(cfg, verbose=True)   # тут ты уже обучал
print("\nSUMMARY:", {k: res[k] for k in ["spikes_per_sample","winners_unique","winner_HHI","energy_proxy_per_sample"]})

# 2) сохраняем после обучения:
save_snn("out/snn_mnist.pt", cfg, connection, lif_layer)

# 3) для оценки — пересобираем сеть (или используем текущую), грузим веса:
net, input_layer, lif_layer, connection, recurrent_inh, W_inh = build_net(cfg)

load_weights_into(net, connection, lif_layer, "out/snn_mnist.pt")

# 4) тот же энкодер, что и при обучении:
encoder, _ = make_encoder("poisson", T=cfg.time)

# 5) калибруем нейрон→метка по train (без обучения!):
label_map = build_label_map(net, input_layer, lif_layer, encoder,  n_calib=2000, T=cfg.time, top_k=cfg.top_k)

# 6) считаем accuracy на test:
test_report = evaluate_on_mnist(net, input_layer, lif_layer, encoder,label_map, T=cfg.time, top_k=cfg.top_k, n_test=1000)
print(test_report)

Используем: cpu
[lif] thresh = scalar 0.350
[lif] tc_decay = scalar 150.000
[lif] refrac = 2
[lif] reset = 0.0
INPUT window sum: 117
LIF   window sum: 0
[1] total=0 active=0/100 HHI=1.000
INPUT window sum: 140
LIF   window sum: 67
[50] total=67 active=13/100 HHI=0.082
INPUT window sum: 79
LIF   window sum: 0
INPUT window sum: 53
LIF   window sum: 0
[100] total=0 active=0/100 HHI=1.000
INPUT window sum: 88
LIF   window sum: 0
INPUT window sum: 165
LIF   window sum: 90
[150] total=90 active=7/100 HHI=0.143
INPUT window sum: 119
LIF   window sum: 11
INPUT window sum: 112
LIF   window sum: 0
[200] total=0 active=0/100 HHI=1.000
INPUT window sum: 183
LIF   window sum: 641
[250] total=641 active=100/100 HHI=0.014
INPUT window sum: 157
LIF   window sum: 357
[300] total=357 active=76/100 HHI=0.018
INPUT window sum: 107
LIF   window sum: 0
[350] total=0 active=0/100 HHI=1.000
INPUT window sum: 224
LIF   window sum: 366
[400] total=366 active=43/100 HHI=0.042
INPUT window sum: 146
LIF   window s